In [2]:
import math
# import os
import sys

from gams.connect import ConnectDatabase

from gamspy import (
    Container,
    Domain,
    Equation,
    Model,
    Number,
    Options,
    Parameter,
    Sense,
    Set,
    Sum,
    Variable,
)
from gamspy.math import Round

m = Container()

# Read from Excel file

In [5]:
# Sets
i = Set(m, name="i", description="rows")
j = Set(m, name="j", description="columns")
k = Set(m, name="k", description="planes")
v = Set(m, name="v", description="non zero cells", domain=[i, j, k])
s = Set(m, name="s", description="sensitive cells", domain=[i, j, k])

# Parameters
dat = Parameter(
    m,
    name="dat",
    description="unprotected data table",
    domain=[k, i, j],
    domain_forwarding=True,
)
pro = Parameter(
    m,
    name="pro",
    description="information sensitive cells",
    domain=[k, i, j],
)

# extract data from Excel
cdb = ConnectDatabase(m.system_directory)
cdb.exec_task(
    {
        "ExcelReader": {
            "file": "./cta.xlsx",
            "symbols": [
                {
                    "name": "dat",
                    "range": "Sheet1!A1",
                    "rowDimension": 2,
                    "columnDimension": 1,
                },
                {
                    "name": "pro",
                    "range": "Sheet2!A1",
                    "rowDimension": 2,
                    "columnDimension": 1,
                },
            ],
        }
    }
)

dat.setRecords(cdb.container["dat"].records)
pro.setRecords(cdb.container["pro"].records)


/Users/ferris/.venvs/gamspy++/lib/python3.12/site-packages/gams/transfer/syms/_methods/tables.py:102: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  records = records.stack(


In [9]:
display(cdb.container["dat"].records)
display(cdb.container["pro"].records)
display(dat.records)
display(pro.records)

,uni_0,uni_1,uni_2,value
0,p1,r1,c1,6764.0
1,p1,r1,c2,714.0
2,p1,r1,c3,3356.0
3,p1,r1,c4,4067.0
4,p1,r1,c5,140.0
...,...,...,...,...
189,total,total,c6,31119.0
190,total,total,c7,17103.0
191,total,total,c8,24537.0
192,total,total,c9,17751.0


,uni_0,uni_1,uni_2,value
0,p1,r1,c2,39.0
1,p1,r1,c5,7.0
2,p1,r2,c9,57.0
3,p1,r5,c8,36.0
4,p2,r1,c2,59.0
5,p2,r1,c4,7.0
6,p2,r2,c4,87.0
7,p2,r2,c6,59.0
8,p2,r3,c6,30.0
9,p2,r3,c7,70.0


,k,i,j,value
0,total,r1,total,46181.0
1,total,r1,c1,6764.0
2,total,r1,c2,4805.0
3,total,r1,c3,6832.0
4,total,r1,c4,4751.0
...,...,...,...,...
186,p3,total,c5,9267.0
187,p3,total,c6,8631.0
188,p3,total,c7,1405.0
189,p3,total,c8,6237.0


,k,i,j,value
0,total,r5,c8,36.0
1,total,r5,c9,88.0
2,p1,r1,c2,39.0
3,p1,r1,c5,7.0
4,p1,r2,c9,57.0
5,p1,r5,c8,36.0
6,p2,r1,c2,59.0
7,p2,r1,c4,7.0
8,p2,r2,c4,87.0
9,p2,r2,c6,59.0


In [8]:
# do some basic data checks
check = Parameter(m, name="check")
check[...] = Sum(
    [i, k], Round(Sum(j, dat[k, i, j]) - 2 * dat[k, i, "total"])
)
assert math.isclose(check.toList()[0], 0), "row totals are incorrect"
check[...] = Sum(
    [j, k], Round(Sum(i, dat[k, i, j]) - 2 * dat[k, "total", j])
)
assert math.isclose(check.toList()[0], 0), "column totals are incorrect"
check[...] = Sum(
    [i, j], Round(Sum(k, dat[k, i, j]) - 2 * dat["total", i, j])
)
assert math.isclose(check.toList()[0], 0), "plane totals are incorrect"
